In [1]:
import json
import os
import glob

base_folder = 'C:/Users/sunxo/Study/BIS/data/data'
json_pattern = os.path.join(base_folder, '**/*.json')
json_files = glob.glob(json_pattern, recursive=True)

img_width = 1920
img_height = 1080

for json_file in json_files:
    with open(json_file, 'r', encoding='utf-8') as file:
        data = json.load(file)

    # 'frame' 키 삭제
    if 'frame' in data:
        del data['frame']

    # 'gender' 값을 0 또는 1로 대응
    if 'gender' in data:
        data['gender'] = 0 if data['gender'] == 'male' else 1

    # 'xtl', 'ytl', 'xbr', 'ybr' 값을 'x_center', 'y_center', 'width', 'height'로 변환 및 정규화
    if all(key in data for key in ['xtl', 'ytl', 'xbr', 'ybr']):
        x_center = float(data['xtl']) / img_width + (float(data['xbr']) - float(data['xtl'])) / (2 * img_width)
        y_center = float(data['ytl']) / img_height + (float(data['ybr']) - float(data['ytl'])) / (2 * img_height)
        width = (float(data['xbr']) - float(data['xtl'])) / img_width
        height = (float(data['ybr']) - float(data['ytl'])) / img_height

        data['x_center'] = x_center
        data['y_center'] = y_center
        data['width'] = width
        data['height'] = height

        del data['xtl']
        del data['ytl']
        del data['xbr']
        del data['ybr']

    # JSON을 txt 파일로 저장
    txt_file = os.path.splitext(json_file)[0] + '.txt'
    with open(txt_file, 'w', encoding='utf-8') as txt_out:
        txt_out.write(f"{data['gender']} {data['x_center']} {data['y_center']} {data['width']} {data['height']}\n")

    # JSON 파일 삭제
    os.remove(json_file)
